# Распознавание именованных сущностей (NER)

Этот ноутбук является частью [Учебной программы по ИИ для начинающих](http://aka.ms/ai-beginners).

В этом примере мы научимся обучать модель NER на [Аннотированном корпусе для распознавания именованных сущностей](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) из Kaggle. Перед началом работы, пожалуйста, скачайте файл [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) в текущую директорию.


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## Подготовка набора данных

Мы начнем с загрузки набора данных в датафрейм. Если вы хотите узнать больше о работе с Pandas, посетите [урок по обработке данных](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) из нашего курса [Data Science для начинающих](http://aka.ms/datascience-beginners).


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


Давайте получим уникальные теги и создадим словари поиска, которые мы можем использовать для преобразования тегов в номера классов:


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

Теперь нам нужно сделать то же самое со словарем. Для простоты мы создадим словарь, не учитывая частоту слов; в реальной жизни вы можете использовать векторизатор Keras и ограничить количество слов.


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

Нам нужно создать набор данных предложений для обучения. Давайте пройдемся по исходному набору данных и разделим все отдельные предложения на `X` (списки слов) и `Y` (списки токенов):


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

Для простоты мы будем дополнять все предложения до максимальной длины 0 токенами. В реальной жизни мы могли бы использовать более умную стратегию и дополнять последовательности только внутри одного минибатча.


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## Определение сети для классификации токенов

Мы будем использовать двухслойную двунаправленную сеть LSTM для классификации токенов. Чтобы применить плотный классификатор к каждому выходу последнего слоя LSTM, мы воспользуемся конструкцией `TimeDistributed`, которая реплицирует один и тот же плотный слой для каждого выхода LSTM на каждом шаге:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

Обратите внимание, что здесь мы явно задаем `maxlen` для нашего набора данных - если мы хотим, чтобы сеть могла обрабатывать последовательности переменной длины, нам нужно быть немного более изобретательными при определении сети.

Теперь давайте обучим модель. Для ускорения мы будем обучать только один эпоху, но вы можете попробовать обучать дольше. Также, возможно, вам стоит выделить часть набора данных как обучающий набор, чтобы наблюдать точность на этапе валидации.


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## Проверка результата

Давайте посмотрим, как наша модель распознавания сущностей работает на примере предложения:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## Вывод

Даже простая модель LSTM показывает неплохие результаты в задаче NER. Однако, чтобы добиться значительно лучших результатов, стоит использовать крупные предварительно обученные языковые модели, такие как BERT. Обучение BERT для NER с использованием библиотеки Huggingface Transformers описано [здесь](https://huggingface.co/course/chapter7/2?fw=pt).



---

**Отказ от ответственности**:  
Этот документ был переведен с использованием сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, имейте в виду, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
